In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfTransformer
from googletrans import Translator
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import re
from tqdm import tqdm
from sklearn.model_selection import KFold
from imblearn.under_sampling import InstanceHardnessThreshold
from sklearn.linear_model import LogisticRegression
import numpy as np
import pandas as pd
import openpyxl
from tqdm import tqdm  
from googletrans import Translator
# Translation
from tqdm import tqdm  
import nltk
nltk.download('stopwords')
nltk.download('punkt')
from nltk.corpus import stopwords
from tqdm import tqdm
import nltk
from nltk.tokenize import word_tokenize
import re
from tqdm import tqdm
from nltk.stem import WordNetLemmatizer
from collections import Counter
from imblearn.under_sampling import RandomUnderSampler, EditedNearestNeighbours, OneSidedSelection, InstanceHardnessThreshold
from sklearn.linear_model import LogisticRegression
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_classification
import numpy as np
from sklearn.model_selection import KFold
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from bidi.algorithm import get_display

In [ ]:
# Load Data
wb = openpyxl.load_workbook('E:/Research_Work/Bangla_CyberBulling_2024/Code & dataset/dataset/Final_Dataset_94k_Bangla_CyberBulling_Combine_Dataset_94k.xlsx')
ws = wb['Sheet1']
data_rows = []
for row in ws['A1':'B94001']:
    data_cols = [cell.value for cell in row]
    data_rows.append(data_cols)
df = pd.DataFrame(data_rows)
header = df.iloc[0]
df.columns = [header]
df = df.iloc[1:, :]

In [ ]:
class_counts = df['label'].value_counts()
print("Class Counts:\n", class_counts)

In [ ]:
all_comments = ' '.join(df['comment'].apply(lambda x: ' '.join(map(str, x))).astype(str))

rgx = r"[\u0980-\u09FF]+"
wordcloud = WordCloud(font_path='E:/Research_Work/Bangla_CyberBulling_2024/Code & dataset/Dataset/Font/CHANO___.ttf',regexp=rgx, width=2000, height=1300, background_color='white').generate(all_comments)

plt.figure(figsize=(30, 15))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis('off')
plt.show()

In [ ]:
comment= df['comment']

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# Applying a function to calculate the number of words in each comment
word_counts = np.vectorize(lambda x: len(str(x).split()))(comment)

plt.figure(figsize=(8, 6))
sns.kdeplot(word_counts, fill=True, color='skyblue')
plt.title('Density Plot of Comments Length Before Preprocessing')
plt.xlabel('Word Count')
plt.ylabel('Density')
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Adjust x-axis ticks at intervals of 5
plt.xticks(np.arange(0, max(word_counts)+1, 15))

# Adjust y-axis ticks to represent percentages
plt.yticks(plt.yticks()[0], [f'{int(t*100)}' for t in plt.yticks()[0]])

plt.show()


In [ ]:
# Data Exploration
print(df.head())
print(df.iloc[:, -1].value_counts())
print(df.shape)
print(df.isna().sum())

# Handle Missing Values
mode = df.iloc[:, -2].value_counts().index[0]
df.iloc[:, -2].fillna(mode, inplace=True)
print(df.isna().sum())
print(df.head())

In [ ]:
# Labeling and Encoding
list(df.iloc[:, -1].value_counts().index)
df['nlabel'] = df.label.replace(['not bully', 'troll', 'sexual', 'religious', 'threat'], [0, 1, 2, 3, 4])
df['noutput'] = df.iloc[:, -1].replace(['bully', 'normal'], [1, 0])
print(df.head())
print(df.head())

In [ ]:
translator = Translator()
output = []

for i in tqdm(range(len(df))):
    if df.label.values[i] == 'not bully':
        output.append('Normal')
    else:
        output.append('Bully')

df['output'] = output
df['noutput'] = df.iloc[:, -1].replace(['Bully', 'Normal'], [1, 0])


In [ ]:
df.head()

In [ ]:
print(stopwords.fileids())
sw = stopwords.words('bengali')

new_stopwords = stopwords.words('english')
new_stopwords.append('SampleWord')

df.to_csv('df_english1000.csv')

nltk.download('wordnet')
nltk.download('omw-1.4')

wordnet_lemmatizer = WordNetLemmatizer()

M = len(df)
for k in tqdm(range(M)):
    text = df.iloc[:, 0].values[k]
    txt = re.sub(r'http\S+', '', text)
    txt = re.sub(r'[!@#$%^&*?><,./\-+`~|:);(❤}{]', '', txt)

    text_tokens = word_tokenize(txt)

    def detect_benglish(text_tokens):
        benglish = False
        for j in range(len(text_tokens)):
            for i in range(len(text_tokens[j])):
                detector = Translator()
                txt = text_tokens[j]
                if i < len(txt) - 1:
                    if detector.detect(txt[i]).lang == 'bn' and detector.detect(txt[i + 1]).lang == 'en' and txt[
                        i].isalpha() == True and txt[i + 1].isalpha() == True:
                        benglish = True
        return benglish

    def handle_benglish(text_tokens):
        for j in range(len(text_tokens)):
            s = []
            for i in range(len(text_tokens[j])):
                detector = Translator()
                txt = text_tokens[j]
                if i < len(txt) - 1:
                    if detector.detect(txt[i]).lang == 'bn' and detector.detect(txt[i + 1]).lang == 'en' and txt[
                        i].isalpha() == True and txt[i + 1].isalpha() == True:
                        s.append(i)
                    if txt[i] == '|':
                        s.append(i)
            for i in range(len(s)):
                txt = txt.replace(txt[s[i]], txt[s[i]] + " ")
            text_tokens[j] = txt
        tstring = str(' '.join(text_tokens)).lower()
        text_tokens = word_tokenize(tstring)
        return text_tokens

    remove_sw = [word for word in text_tokens if not word in sw]
    un_items = np.unique(remove_sw)
    r_sw = [wordnet_lemmatizer.lemmatize(w) for w in un_items]
    bn_tokens = []

    def tanslate_bengali(r_sw):
        for i in range(len(r_sw)):
            bn_tokens.append(translator.translate(r_sw[i], dest='bn').text)
        return bn_tokens

    bn_token = r_sw
    df.iloc[:, 0].values[k] = ' '.join(bn_token)

df.iloc[:, -1].value_counts()
df.iloc[:, 0]

df[:].iloc[:, 0].shape


In [ ]:
comment = df['comment']

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Applying a function to calculate the number of words in each comment
word_counts = np.vectorize(lambda x: len(str(x).split()))(comment)

plt.figure(figsize=(8, 6))
sns.kdeplot(word_counts, shade=True, color='coral')  # Use 'coral' directly
plt.title('Density Plot of Comments Length After Preprocessing')
plt.xlabel('Word Count')
plt.ylabel('Density')
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Adjust x-axis ticks at intervals of 5
plt.xticks(np.arange(0, max(word_counts)+1, 10))

# Adjust y-axis ticks to represent percentages
plt.yticks(plt.yticks()[0], [f'{int(t*100)}' for t in plt.yticks()[0]])

plt.show()

In [ ]:
# Text Vectorization (Count)
count_vectorizer = CountVectorizer(max_features=20000, tokenizer=word_tokenize, stop_words=stopwords.words('english'))
iX_count = count_vectorizer.fit_transform(df[:].iloc[:, 0].values).toarray()

In [ ]:
tfidf_vectorizer = TfidfTransformer()

tf_transformer = TfidfTransformer(use_idf=True).fit(iX_count)
iX_tfidf = tf_transformer.transform(iX_count)

iX_tfidf.shape

In [ ]:
by = df.iloc[:, -2].values
by.shape

my = df.iloc[:, -1].values

np.unique(by)

In [ ]:
# Imbalanced Data Handling
cc = InstanceHardnessThreshold(random_state=10, estimator=LogisticRegression())
X, y = cc.fit_resample(iX_tfidf, by)
X.shape, y.shape

In [ ]:
# Cross-Validation
kf = KFold(n_splits=5, random_state=42, shuffle=True)
for train_index, test_index in kf.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
print('complete')
print('train:', X_train.shape, y_train.shape, 'test:', X_test.shape, y_test.shape)

In [ ]:
# SGD+LR+MLP (Stacking)

In [ ]:
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.base import is_classifier

# Assuming you have X_train, X_test, y_train, y_test defined somewhere in your code

# Part 51: Model Building (Stacking Classifier: SGD, MLP, RandomForest)
def build_stacking_classifier(X_train, y_train):
    global stacking_model
    print('Building Stacking Classifier model...')
    
    # Creating instances of the base classifiers
    sgd_classifier = SGDClassifier()
    mlp_classifier = MLPClassifier()
    rf_classifier = RandomForestClassifier()

    # Check if classifiers support probability estimates
    classifiers_with_proba = [(name, clf) for name, clf in [('sgd', sgd_classifier), ('mlp', mlp_classifier), ('rf', rf_classifier)] if is_classifier(clf) and hasattr(clf, "predict_proba")]

    if not classifiers_with_proba:
        raise ValueError("None of the base classifiers support probability estimates. Stacking requires probability estimates.")

    # Creating the Stacking Classifier
    stacking_model = StackingClassifier(classifiers_with_proba, final_estimator=RandomForestClassifier(), stack_method='auto', n_jobs=-1)
    
    # Fitting the model
    stacking_model.fit(X_train, y_train)
    
    print('Stacking Classifier model built successfully.')
    return stacking_model

# Building the Stacking Classifier (assuming X_train, X_test, y_train, y_test are defined)
stacking_model = build_stacking_classifier(X_train, y_train)

# Part 52: Model Evaluation (Stacking Classifier)
def evaluate_stacking_classifier(model, X_test, y_test):
    y_pred = model.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    print(f'Accuracy: {acc*100:.2f}%')
    
    # Additional metrics
    precision = precision_score(y_test, y_pred, average='weighted')
    recall = recall_score(y_test, y_pred, average='weighted')
    f1 = f1_score(y_test, y_pred, average='weighted')
    
    print(f'Precision: {precision*100:.2f}%')
    print(f'Recall: {recall*100:.2f}%')
    print(f'F1 Score: {f1*100:.2f}%')
    
    confusion_matrix_analysis(model, X_test, y_test)
    
    return acc

# Part 53: Confusion Matrix Analysis (Stacking Classifier)
def confusion_matrix_analysis(model, X_test, y_test):
    y_pred = model.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)
    class_names = ['Normal', 'Bully']  # Update class names
    
    plt.figure(figsize=(4, 3))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Greens", xticklabels=class_names, yticklabels=class_names)
    
    # Display values on the plot without box and color
    for i in range(len(class_names)):
        for j in range(len(class_names)):
            plt.text(j + 0.5, i + 0.5, f'{cm[i, j]}', ha='center', va='center', color='black', fontsize=500)

    plt.title('Confusion Matrix (Stacking Classifier)')
    plt.xlabel('Predicted label')
    plt.ylabel('True label')
    plt.show()

# Evaluating the Stacking Classifier
accuracy_stacking = evaluate_stacking_classifier(stacking_model, X_test, y_test)


In [ ]:

# Part 53: Confusion Matrix Analysis (Stacking Classifier)
def confusion_matrix_analysis(model, X_test, y_test):
    y_pred = model.predict(X_test)
    cm = confusion_matrix(y_test, y_pred)
    class_names = np.unique(y_train) # Update class names
    
    plt.figure(figsize=(4, 3))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Greens", xticklabels=class_names, yticklabels=class_names)
    
    # Display values on the plot without box and color
    for i in range(len(class_names)):
        for j in range(len(class_names)):
            plt.text(j + 0.5, i + 0.5, f'{cm[i, j]}', ha='center', va='center', color='black', fontsize=10, weight='bold')

    plt.title('Confusion Matrix (Stacking Classifier)')
    plt.xlabel('Predicted label')
    plt.ylabel('True label')
    plt.show()

# Evaluating the Stacking Classifier
accuracy_stacking = evaluate_stacking_classifier(stacking_model, X_test, y_test)

In [ ]:
from sklearn.metrics import roc_curve, auc
# Part 54: ROC Curve (LightGBM with Default Parameters)
def plot_roc_curve(model, X_test, y_test, class_names):
    y_probs = model.predict_proba(X_test)

    plt.figure(figsize=(4, 3))
    for i in range(len(class_names)):
        fpr, tpr, thresholds = roc_curve(y_test, y_probs[:, i], pos_label=class_names[i])
        roc_auc = auc(fpr, tpr)
        plt.plot(fpr, tpr, label=f'{class_names[i]} (AUC = {roc_auc:.2f})')

    plt.plot([0, 1], [0, 1], linestyle='--', color='grey', label='Random')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curve')
    plt.legend()
    plt.show()

class_names = np.unique(y_train)
plot_roc_curve(stacking_model, X_test, y_test, class_names)


In [ ]:
from sklearn.model_selection import cross_val_score

# Part 54: Cross-Validation (Stacking Classifier)
def cross_validate_stacking_classifier(model, X, y, cv=5):
    print(f'Cross-validating Stacking Classifier with {cv} folds...')
    
    # Using cross_val_score for cross-validation
    scores = cross_val_score(model, X, y, cv=cv, scoring='accuracy')
    
    # Displaying cross-validation results
    print(f'Cross-Validation Accuracy: {scores.mean()*100:.2f}% (±{scores.std()*100:.2f}%)')
    
    return scores

# Cross-validating the Stacking Classifier
cv_scores = cross_validate_stacking_classifier(stacking_model, X_train, y_train)
